In [2]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn

In [77]:
df = pd.read_parquet('data/train_transactions/part_000_0_to_23646.parquet')

In [79]:
head = df.head(5)

In [80]:
head.to_csv('head.csv', index=False)

In [54]:
data = pd.DataFrame({'feature1': [1, 2, 2], 'feature2': [4, 5, 6]})

In [67]:
num_categories = 2
feat1 = data['feature1']
embedding_dim = 3
embedding_layer = nn.Embedding(num_categories, embedding_dim)

In [64]:
feat1 = torch.tensor(feat1)
feat1

tensor([1, 2, 2])

In [65]:
emb = embedding_layer(feat1)

In [71]:
import pandas as pd
import torch
import torch.nn as nn
from sklearn.preprocessing import LabelEncoder

# Пример DataFrame с категориальными признаками
data = pd.DataFrame({
    'feature1': ['A', 'B', 'A', 'C', 'B', 'A'],
    'feature2': ['X', 'X', 'Y', 'Z', 'Y', 'X']
})

# Шаг 1: Кодирование категориальных признаков с помощью LabelEncoder
label_encoders = {}
encoded_data = pd.DataFrame()
for column in data.columns:
    le = LabelEncoder()
    encoded_data[column] = le.fit_transform(data[column])
    label_encoders[column] = le

# Преобразуем в тензор
encoded_tensor = torch.tensor(encoded_data.values, dtype=torch.long)

# Шаг 2: Создание эмбеддинг слоев для каждого категориального признака
embedding_dims = {
    'feature1': 4,  # Размерность эмбеддинга для feature1
    'feature2': 4   # Размерность эмбеддинга для feature2
}

embedding_layers = {
    'feature1': nn.Embedding(num_embeddings=len(label_encoders['feature1'].classes_), embedding_dim=embedding_dims['feature1']),
    'feature2': nn.Embedding(num_embeddings=len(label_encoders['feature2'].classes_), embedding_dim=embedding_dims['feature2'])
}

# Шаг 3: Применение эмбеддингов
embeddings = []
for i, feature in enumerate(embedding_layers):
    embedding = embedding_layers[feature](encoded_tensor[:, i])
    embeddings.append(embedding)

# Объединение эмбеддингов в один тензор (можно использовать при подаче в модель)
final_embeddings = torch.cat(embeddings, dim=1)

# Вывод результата
print("Закодированные категориальные признаки:\n", encoded_tensor)
print("\nЭмбеддинги:\n", final_embeddings)

Закодированные категориальные признаки:
 tensor([[0, 0],
        [1, 0],
        [0, 1],
        [2, 2],
        [1, 1],
        [0, 0]])

Эмбеддинги:
 tensor([[-1.0131,  1.1354, -0.6742, -2.0086, -0.3562,  0.1584,  0.5514,  0.2430],
        [-1.5023,  1.0045,  0.3310, -0.4085, -0.3562,  0.1584,  0.5514,  0.2430],
        [-1.0131,  1.1354, -0.6742, -2.0086,  0.8777,  0.5031, -0.7266,  0.0386],
        [-0.4568,  0.6416, -1.2689,  0.9754,  0.5659, -1.0960,  0.6948,  0.2340],
        [-1.5023,  1.0045,  0.3310, -0.4085,  0.8777,  0.5031, -0.7266,  0.0386],
        [-1.0131,  1.1354, -0.6742, -2.0086, -0.3562,  0.1584,  0.5514,  0.2430]],
       grad_fn=<CatBackward0>)


In [72]:
label_encoders['feature1'].classes_

array(['A', 'B', 'C'], dtype=object)

In [69]:
final_embeddings.shape

torch.Size([6, 8])

In [73]:
pip install pytorch-lifestream

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 163.4/163.4 kB 1.1 MB/s eta 0:00:00a 0:00:01
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.2/12.2 MB 3.1 MB/s eta 0:00:0000:0100:01
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Using cached numpy-1.24.4-cp38-cp38-macosx_10_9_x86_64.whl.metadata (5.6 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.0/117.0 kB 7.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.5/154.5 kB 3.1 MB/s eta 0:00:00a 0:00:01
Using cached numpy-1.24.4-cp38-cp38-macosx_10_9_x86_64.whl (19.8 MB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.5/79.5 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 815.2/815.2 kB 2.9 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━